In [2]:
import io
import os
import sys
import types
import numpy as np
import pandas as pd
import altair as alt
import dropbox

In [3]:
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [4]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [5]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [6]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [7]:
sys.meta_path.append(NotebookFinder())

In [8]:
from datetime import date
from datetime import datetime, timedelta
import knmi_gn_001 as gethdd

importing Jupyter notebook from knmi_gn_001.ipynb


In [9]:
def RetrieveDailyData(fn):
    # lees de file fn in
    try:
        df = pd.read_csv(fn,
                       parse_dates=['datetime'],
                       index_col = ['datetime'])
    except:
        print("File not found ",fn)
    
    return df

In [10]:
fn = 'gas_consumption_day.csv'
df = RetrieveDailyData(fn)
df.tail()


,m3_cv,m3_tap
datetime,,
2021-10-18,2.865800,0.381200
2021-10-19,1.900000,0.298000
2021-10-20,0.915000,0.857000
2021-10-21,2.281267,0.659733
2021-10-22,3.600300,0.354700


In [11]:
df = df[["m3_cv","m3_tap"]]
df.describe()

,m3_cv,m3_tap
count,991.000000,991.000000
mean,4.564558,6.177348
std,48.018188,120.220906
min,0.000000,0.000000
25%,0.000000,0.436500
50%,1.576000,0.635000
75%,5.394617,0.872300
max,1510.259250,2784.381000


In [12]:
# Get the knmi data daily granularity
df_hdd = gethdd.retrieveHDD()
df_hdd = df_hdd[df.index.min():]
df = df.merge(df_hdd,left_on="datetime",right_on="datetime",how='right')

read the csv file with knmi data  knmi_data.csv
last data point in csv file:  2021-10-22 00:00:00
The current time is : 2021-10-23 11:18:32.528591
The number of days since last datapoint is : 1
KNMI last datapoint is : 2021-10-22 00:00:00
Current time : 2021-10-23 11:18:32.571314
Elapsed time since  [hours] :  35
Nr rows to add :  1
              SQ      Q    DR     TG    FG        HDD        dow
datetime                                                        
2021-10-19   7.0  212.0  10.0  157.0  37.0   0.766667    Tuesday
2021-10-20   5.0  267.0  28.0  157.0  48.0   1.500000  Wednesday
2021-10-21  32.0  463.0  89.0   97.0  50.0   7.633333   Thursday
2021-10-22  31.0  425.0  16.0   79.0  46.0   9.166667     Friday
2021-10-23   0.0    0.0   0.0    0.0   0.0  14.000000   Saturday


In [13]:
df = df[df['m3_cv'].isnull()==False]
df.tail()

,m3_cv,m3_tap,SQ,Q,DR,TG,FG,HDD,dow
datetime,,,,,,,,,
2021-10-18,2.865800,0.381200,32.0,632.0,0.0,118.0,25.0,3.866667,Monday
2021-10-19,1.900000,0.298000,7.0,212.0,10.0,157.0,37.0,0.766667,Tuesday
2021-10-20,0.915000,0.857000,5.0,267.0,28.0,157.0,48.0,1.500000,Wednesday
2021-10-21,2.281267,0.659733,32.0,463.0,89.0,97.0,50.0,7.633333,Thursday
2021-10-22,3.600300,0.354700,31.0,425.0,16.0,79.0,46.0,9.166667,Friday


In [14]:
df['period'] ="PreCovid"
df.loc[df.index>"2021-01-01","period"]="Covid"
df.tail()

,m3_cv,m3_tap,SQ,Q,DR,TG,FG,HDD,dow,period
datetime,,,,,,,,,,
2021-10-18,2.865800,0.381200,32.0,632.0,0.0,118.0,25.0,3.866667,Monday,Covid
2021-10-19,1.900000,0.298000,7.0,212.0,10.0,157.0,37.0,0.766667,Tuesday,Covid
2021-10-20,0.915000,0.857000,5.0,267.0,28.0,157.0,48.0,1.500000,Wednesday,Covid
2021-10-21,2.281267,0.659733,32.0,463.0,89.0,97.0,50.0,7.633333,Thursday,Covid
2021-10-22,3.600300,0.354700,31.0,425.0,16.0,79.0,46.0,9.166667,Friday,Covid


In [15]:
titleshown = "CV vs Graaddagen"
x_max = 30
y_max = 20
pw = 600
ph = 400
source = df[df['m3_cv']>0.1]
chart_hdd = alt.Chart(source,title=titleshown).mark_point(clip=True,color="lightgrey").encode(
    x=alt.X('HDD:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("m3_cv:Q",  scale=alt.Scale(domain=(0, y_max)))
).properties(width=pw,height=ph)


base = alt.Chart(source).mark_point(clip=True,size=100).encode(
    x=alt.X('HDD:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("m3_cv:Q",  scale=alt.Scale(domain=(0, y_max))))

points = base.mark_point(clip=True,size=100).encode(
    fill = alt.Fill('period:N', scale = alt.Scale(scheme='plasma'))
)


chart_hdd = chart_hdd + points 


In [16]:
chart_hdd

alt.LayerChart(...)

In [16]:
df.groupby(['period']).size()

period
Covid       116
PreCovid    715
dtype: int64

In [19]:
df[(df['HDD']>9.8)&(df['HDD']<10.2)]

,m3_cv,m3_tap,SQ,Q,DR,TG,FG,HDD,dow,period
datetime,,,,,,,,,,
2019-01-06,5.317200,2.743800,11.0,154.0,54.0,58.0,28.0,10.066667,Sunday,PreCovid
2019-01-13,7.659000,0.000000,0.0,58.0,85.0,84.0,65.0,9.933333,Sunday,PreCovid
2019-02-08,8.172400,0.591600,0.0,129.0,20.0,79.0,58.0,9.966667,Friday,PreCovid
2019-02-19,6.192600,0.919400,39.0,541.0,0.0,69.0,43.0,9.966667,Tuesday,PreCovid
2019-03-19,2.875800,0.307200,85.0,1332.0,0.0,52.0,16.0,9.866667,Tuesday,PreCovid
2020-01-07,7.118600,0.805400,28.0,239.0,0.0,69.0,42.0,9.900000,Tuesday,PreCovid
2020-02-09,4.696400,0.918600,0.0,169.0,45.0,100.0,88.0,9.866667,Sunday,PreCovid
2020-02-20,4.341900,0.805100,15.0,401.0,59.0,77.0,58.0,10.166667,Thursday,PreCovid
2020-03-14,4.888800,1.243200,73.0,1202.0,0.0,61.0,30.0,9.900000,Saturday,PreCovid


In [21]:
titleshown = "CV vs Graaddagen"
x_max = 30
y_max = 20
pw = 600
ph = 400
source = df[(df['HDD']>9.8)&(df['HDD']<10.2)]
chart_hdd = alt.Chart(source,title=titleshown).mark_point(clip=True,color="lightgrey").encode(
    x=alt.X('HDD:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("m3_cv:Q",  scale=alt.Scale(domain=(0, y_max)))
).properties(width=pw,height=ph)


base = alt.Chart(source).mark_point(clip=True,size=100).encode(
    x=alt.X('HDD:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("m3_cv:Q",  scale=alt.Scale(domain=(0, y_max))))

points = base.mark_point(clip=True,size=100).encode(
    fill = alt.Fill('period:N', scale = alt.Scale(scheme='plasma'))
)


chart_hdd = chart_hdd + points 
chart_hdd

alt.LayerChart(...)

In [33]:
titleshown = "CV vs DR"
x_max = 30
y_max = 20
pw = 600
ph = 400
source = df[(df['HDD']>9.8)&(df['HDD']<10.2)]
chart_hdd = alt.Chart(source,title=titleshown).mark_point(clip=True,color="lightgrey").encode(
    x=alt.X('DR:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("m3_cv:Q",  scale=alt.Scale(domain=(0, y_max)))
).properties(width=pw,height=ph)


base = alt.Chart(source).mark_point(clip=True,size=100).encode(
    x=alt.X('DR:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("m3_cv:Q",  scale=alt.Scale(domain=(0, y_max))))

points = base.mark_point(clip=True,size=100).encode(
    fill = alt.Fill('period:N', scale = alt.Scale(scheme='plasma'))
)


chart_hdd = chart_hdd + points 
chart_hdd

alt.LayerChart(...)

Vergelijk 2 datapunten:

2021-02-16 & 2021-04-04

In [23]:
date1 = '2021-02-16'
date2 = '2021-04-04'
dates = [date1,date2]

In [30]:
df[df.index.isin(dates)]

,m3_cv,m3_tap,SQ,Q,DR,TG,FG,HDD,dow,period
datetime,,,,,,,,,,
2021-02-16,11.7866,0.7214,15.0,467.0,26.0,63.0,33.0,9.900000,Tuesday,Covid
2021-04-04,4.1604,0.6246,23.0,804.0,0.0,61.0,31.0,9.966667,Sunday,Covid


Vergelijk ruwe data punten

In [34]:
import GetMySQL

importing Jupyter notebook from GetMySQL.ipynb


In [125]:
HDDmin = 19
HDDmax = 30

In [126]:
dates = df[(df['HDD']>HDDmin)&(df['HDD']<HDDmax)].index

In [127]:
df[df.index.isin(dates)]

,m3_cv,m3_tap,SQ,Q,DR,TG,FG,HDD,dow,period
datetime,,,,,,,,,,
2019-01-24,15.846000,0.000000,0.0,191.0,0.0,-45.0,15.0,19.500000,Thursday,PreCovid
2021-02-07,16.288333,1.994667,0.0,177.0,171.0,-47.0,82.0,24.166667,Sunday,Covid
2021-02-08,17.607000,0.920000,0.0,279.0,43.0,-61.0,44.0,23.033333,Monday,Covid
2021-02-09,15.284200,0.443800,59.0,622.0,0.0,-54.0,18.0,20.600000,Tuesday,Covid
2021-02-10,15.855100,0.636900,61.0,722.0,0.0,-36.0,22.0,19.066667,Wednesday,Covid
2021-02-11,13.856400,1.176600,87.0,990.0,0.0,-63.0,6.0,20.700000,Thursday,Covid
2021-02-12,16.041950,1.245050,87.0,957.0,0.0,-71.0,31.0,23.166667,Friday,Covid
2021-02-13,16.554733,0.608267,89.0,1101.0,0.0,-82.0,28.0,24.066667,Saturday,Covid
2021-02-14,17.577800,0.959200,88.0,932.0,0.0,-29.0,35.0,19.233333,Sunday,Covid


In [128]:
titleshown = "CV vs Graaddagen"
x_max = 30
y_max = 20
pw = 400
ph = 300
source = df[df.index.isin(dates)]
chart_hdd = alt.Chart(source,title=titleshown).mark_point(clip=True,color="lightgrey").encode(
    x=alt.X('HDD:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("m3_cv:Q",  scale=alt.Scale(domain=(0, y_max)))
).properties(width=pw,height=ph)


base = alt.Chart(source).mark_point(clip=True,size=100).encode(
    x=alt.X('HDD:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("m3_cv:Q",  scale=alt.Scale(domain=(0, y_max))))
points = base.mark_point(clip=True,size=100).encode(
    fill = alt.Fill('DR:Q', scale = alt.Scale(scheme='plasma'))
)

chart_hdd = chart_hdd + points 
chart_hdd

alt.LayerChart(...)

In [129]:
df[df.index.isin(dates)].index[0].strftime("%Y-%m-%d")

'2019-01-24'

In [130]:
pct = 90
poi1 = df[df.index.isin(dates)].index[5]
poi2 = df[df.index.isin(dates)].index[5] + pd.Timedelta(hours=24)

poi =''.join( [poi1.strftime("%Y-%m-%d"),':',
               poi2.strftime("%Y-%m-%d")])
poi = GetMySQL.SetPOI(poi)
poi
GetMySQL.retrieveGasandTemps(poi,pct)

POI is :  {'start': '2021-02-11', 'end': '2021-02-12'}
All Datapoints do meet treshold
All Datapoints do meet treshold
All Datapoints do meet treshold
Empty DataFrame
Columns: []
Index: []
No extreme values detected [datetime.date(2021, 2, 11)]


,abs_CVout,abs_gas,abs_tapwater,diff_CVout,diff_gas,diff_tapwater
datetime,,,,,,
2021-02-11 00:00:00,28.56,3804.074,31.69,0.00,0.0,0.00
2021-02-11 00:01:00,28.56,3804.074,31.69,0.00,0.0,0.00
2021-02-11 00:02:00,28.56,3804.074,31.69,0.00,0.0,0.00
2021-02-11 00:03:00,28.50,3804.074,31.31,-0.06,0.0,-0.38
2021-02-11 00:04:00,28.50,3804.074,31.31,0.00,0.0,0.00
...,...,...,...,...,...,...
2021-02-11 23:56:00,29.75,3819.220,29.31,-0.06,0.0,-0.07
2021-02-11 23:57:00,29.75,3819.220,29.31,0.00,0.0,0.00
2021-02-11 23:58:00,NaN,3819.220,NaN,NaN,0.0,NaN


In [131]:
pct = 90
rd = []
for d in dates:
    print(d)
    poi1 = d
    poi2 = d + pd.Timedelta(hours=24)
    poi =''.join( [poi1.strftime("%Y-%m-%d"),':',
               poi2.strftime("%Y-%m-%d")])
    poi = GetMySQL.SetPOI(poi)
    dummy = GetMySQL.retrieveGasandTemps(poi,pct)
    dummy['rowid'] = np.arange(len(dummy))
    rd.append(dummy)
rd = pd.concat(rd)

2019-01-24 00:00:00
POI is :  {'start': '2019-01-24', 'end': '2019-01-25'}
All Datapoints do meet treshold
All Datapoints do meet treshold
All Datapoints do meet treshold
Empty DataFrame
Columns: []
Index: []
No extreme values detected [datetime.date(2019, 1, 24)]
2021-02-07 00:00:00
POI is :  {'start': '2021-02-07', 'end': '2021-02-08'}
All Datapoints do meet treshold
All Datapoints do meet treshold
All Datapoints do meet treshold
Empty DataFrame
Columns: []
Index: []
Anomalies removed : DatetimeIndex(['2021-02-07 03:59:00'], dtype='datetime64[ns]', name='datetime', freq=None)
2021-02-08 00:00:00
POI is :  {'start': '2021-02-08', 'end': '2021-02-09'}
All Datapoints do meet treshold
All Datapoints do meet treshold
All Datapoints do meet treshold
Empty DataFrame
Columns: []
Index: []
No extreme values detected [datetime.date(2021, 2, 8)]
2021-02-09 00:00:00
POI is :  {'start': '2021-02-09', 'end': '2021-02-10'}
All Datapoints do meet treshold
All Datapoints do meet treshold
All Datapoin

In [132]:
rd.tail()

,abs_CVout,abs_gas,abs_tapwater,diff_CVout,diff_gas,diff_tapwater,rowid
datetime,,,,,,,
2021-02-14 23:55:00,36.06,3872.31,33.56,-0.13,0.0,-0.07,1436
2021-02-14 23:56:00,35.94,3872.31,33.44,-0.12,0.0,-0.12,1437
2021-02-14 23:57:00,35.81,3872.31,33.38,-0.13,0.0,-0.06,1438
2021-02-14 23:58:00,35.69,3872.31,33.31,-0.12,0.0,-0.07,1439
2021-02-14 23:59:00,35.63,3872.31,33.25,-0.06,0.0,-0.06,1440


In [133]:
rd['dag'] = rd.index.strftime('%Y-%m-%d')
rd['yr'] = rd.index.year

In [134]:
titleshown = "Temp CV out"
x_max = 1450
y_max = 80
pw = 300
ph = 200
source = rd
chart_temp = alt.Chart(source.sample(5000),title=titleshown).mark_line(clip=True).encode(
    x=alt.X('rowid:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("abs_CVout:Q",  scale=alt.Scale(domain=(0, y_max))),
    color = alt.Color('dag:N')
).properties(width=pw,height=ph).facet(
    facet='yr:N',
    columns=2
)

 
chart_temp

alt.FacetChart(...)

In [94]:
rd['hr'] = rd.index.hour

In [97]:
hraverage  = rd.groupby(by=['dag','hr'])['abs_CVout'].mean().reset_index()

In [98]:
hraverage

,dag,hr,abs_CVout
0,2019-01-06,0,26.179500
1,2019-01-06,1,25.489500
2,2019-01-06,2,24.894167
3,2019-01-06,3,24.339833
4,2019-01-06,4,23.833051
...,...,...,...
499,2021-04-15,19,42.730333
500,2021-04-15,20,34.245333
501,2021-04-15,21,45.795833
502,2021-04-15,22,38.858833


In [99]:
titleshown = "Temp CV out"
x_max = 23
y_max = 80
pw = 1000
ph = 500
source = hraverage
chart_temp = alt.Chart(source,title=titleshown).mark_line(clip=True).encode(
    x=alt.X('hr:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("abs_CVout:Q",  scale=alt.Scale(domain=(0, y_max))),
    color = alt.Color('dag:N')
).properties(width=pw,height=ph)

 
chart_temp

alt.Chart(...)

In [107]:
hraverage['cumul'] = hraverage.groupby(by=['dag'])['abs_CVout'].cumsum()

In [110]:
titleshown = "Temp CV out"
x_max = 23
y_max = 80*15
pw = 1000
ph = 500
source = hraverage
chart_temp = alt.Chart(source,title=titleshown).mark_line(clip=True).encode(
    x=alt.X('hr:Q', scale = alt.Scale(domain=(0,x_max))),
    y=alt.Y("cumul:Q",  scale=alt.Scale(domain=(0, y_max))),
    color = alt.Color('dag:N')
).properties(width=pw,height=ph)

 
chart_temp

alt.Chart(...)

In [136]:
# Neerslag
df.head()

,m3_cv,m3_tap,SQ,Q,DR,TG,FG,HDD,dow,period
datetime,,,,,,,,,,
2019-01-01,5.342550,1.530450,20.0,183.0,27.0,71.0,51.0,10.300000,Tuesday,PreCovid
2019-01-02,6.517000,1.447000,43.0,299.0,0.0,44.0,30.0,11.600000,Wednesday,PreCovid
2019-01-03,7.915400,0.708600,24.0,247.0,3.0,36.0,18.0,11.600000,Thursday,PreCovid
2019-01-04,7.070667,1.999333,0.0,40.0,9.0,49.0,34.0,11.366667,Friday,PreCovid
2019-01-05,6.819400,1.206600,0.0,95.0,27.0,72.0,44.0,9.733333,Saturday,PreCovid


In [140]:
df.sort_values('Q',ascending = False).head(20)

,m3_cv,m3_tap,SQ,Q,DR,TG,FG,HDD,dow,period
datetime,,,,,,,,,,
2021-06-01,0.000,0.443,152.0,3039.0,0.0,177.0,29.0,0.000000,Tuesday,Covid
2020-06-24,0.000,0.432,155.0,3006.0,0.0,210.0,25.0,0.000000,Wednesday,PreCovid
2020-05-29,0.000,0.951,150.0,2997.0,0.0,141.0,24.0,1.500000,Friday,PreCovid
2021-06-07,0.000,0.522,154.0,2982.0,0.0,172.0,20.0,0.000000,Monday,Covid
2021-06-13,0.000,0.713,151.0,2976.0,0.0,159.0,12.0,0.000000,Sunday,Covid
2021-05-30,0.000,0.834,149.0,2955.0,0.0,139.0,16.0,1.166667,Sunday,Covid
2020-05-28,0.000,0.690,134.0,2937.0,0.0,130.0,27.0,2.800000,Thursday,PreCovid
2021-06-14,0.000,0.522,154.0,2937.0,0.0,199.0,20.0,0.000000,Monday,Covid
2020-05-26,0.000,0.385,134.0,2933.0,0.0,143.0,10.0,0.366667,Tuesday,PreCovid
